In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc
import json

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Para ejecutar estos análisis primero se tiene que ejecutar el notebook del archivo "datos_historicos.ipynb", ese notebook permite descargar los archivos de datos con los que trabaja este notebook.

In [3]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "07/09/2020"

In [4]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [5]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [6]:
flag = 0
for fecha in fechas_csv:
    data = pd.read_csv('./datos_federales/' + fecha,encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    if flag == 0:
        all_data = data
        flag = 1
    else:
        all_data = pd.concat([all_data,data],axis=0,ignore_index=True)

TypeError: '_LocIndexer' object does not support item assignment

In [7]:
len(all_data)

25087145

Conversión de fechas a formato de fecha

In [8]:
#all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
#all_data["FECHA_ARCHIVO_c"] = all_data["FECHA_ARCHIVO"].dt.strftime(date_format="%d-%m")
#all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
#all_data["FECHA_ACTUALIZACION_c"] = all_data["FECHA_ACTUALIZACION"].dt.strftime(date_format="%d-%m")
#all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
#all_data["FECHA_INGRESO_c"] = all_data["FECHA_INGRESO"].dt.strftime(date_format="%d-%m")
#all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
#all_data["FECHA_DEF_c"] = all_data["FECHA_DEF"].dt.strftime(date_format="%d-%m")
#all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")
#all_data["FECHA_SINTOMAS_c"] = all_data["FECHA_SINTOMAS"].dt.strftime(date_format="%d-%m")

In [9]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")
all_data["ENTIDAD_UM"] = all_data["ENTIDAD_UM"].astype(str).str.zfill(2)
all_data["ENTIDAD_NAC"] = all_data["ENTIDAD_NAC"].astype(str).str.zfill(2)
all_data["ENTIDAD_RES"] = all_data["ENTIDAD_RES"].astype(str).str.zfill(2)
all_data["MUNICIPIO_RES"] = all_data["MUNICIPIO_RES"].fillna(0).astype(int).astype(str).str.zfill(3)
all_data["CVEGEO_RES"] = all_data["ENTIDAD_RES"] + all_data["MUNICIPIO_RES"]

In [10]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO', 'CVEGEO_RES'],
      dtype='object')

In [10]:
confirmados = all_data[all_data["RESULTADO"]==1]
difuntos = confirmados[confirmados["FECHA_DEF"].notna()]


KeyboardInterrupt



In [ ]:
pacientes_mun = confirmados.groupby(["ID_REGISTRO","CVEGEO_RES","ENTIDAD_RES","MUNICIPIO_RES"]).size().reset_index().drop(columns=[0])
pacientes_edo = confirmados.groupby(["ID_REGISTRO","ENTIDAD_RES"]).size().reset_index().drop(columns=[0])

In [ ]:
varios_mun = pacientes["ID_REGISTRO"].value_counts()[pacientes["ID_REGISTRO"].value_counts()>1].index
varios_edo = pacientesedo["ID_REGISTRO"].value_counts()[pacientesedo["ID_REGISTRO"].value_counts()>1].index

In [ ]:
confirmados[:3]

In [ ]:
def geomatriz(frame,fecha_sel,pacientes_edo,varios_edo,pacientes_mun,varios_mun):
    por_fecha = frame.groupby(["ID_REGISTRO",fecha_sel]).size().reset_index().pivot(index=fecha_sel,columns="ID_REGISTRO",values=0).sort_index().T
    fechas_validas = por_fecha.columns
    validos_mun = pd.merge(pacientes_mun,por_fecha.reset_index()[~por_fecha.reset_index()["ID_REGISTRO"].isin(varios_mun)],how="right",on="ID_REGISTRO")
    validos_edo = pd.merge(pacientes_edo,por_fecha.reset_index()[~por_fecha.reset_index()["ID_REGISTRO"].isin(varios_edo)],how="right",on="ID_REGISTRO")
    validos_edo["ENTIDAD_RES"] = validos_edo["ENTIDAD_RES"].str.zfill(5)
    edo_frame = validos_edo.groupby("ENTIDAD_RES")[fechas_validas].sum().T
    mun_frame = validos_mun.groupby("CVEGEO_RES")[fechas_validas].sum().T
    nac_frame = por_fecha.sum().to_frame("00000")
    return pd.concat([nac_frame,edo_frame,mun_frame],axis=1)

In [ ]:
casos = geomatriz(confirmados,"FECHA_ARCHIVO",pacientes_edo,varios_edo,pacientes_mun,varios_mun)

In [ ]:
casos[-3:]

In [ ]:
casos.to_csv("./datos_procesados/casos.csv")

In [ ]:
defunciones = geomatriz(difuntos,"FECHA_ARCHIVO",pacientes_edo,varios_edo,pacientes_mun,varios_mun)

In [ ]:
defunciones[-3:]

In [ ]:
defunciones.to_csv("./datos_procesados/defunciones.csv")

In [ ]:
fechas = confirmados["FECHA_ARCHIVO"].sort_values().unique()

In [ ]:
for fecha in fechas:
    fecha_pan = pd.to_datetime(fecha)
    filtrados = confirmados[confirmados["FECHA_ARCHIVO"]==fecha_pan]
    matriz_filt = geomatriz(filtrados,"FECHA_SINTOMAS",pacientes_edo,varios_edo,pacientes_mun,varios_mun)
    matriz_filt.cumsum().to_csv("./datos_procesados/casos_sint_" + fecha_pan.strftime("%d-%m") + ".csv")

In [ ]:
for fecha in fechas:
    fecha_pan = pd.to_datetime(fecha)
    filtrados = difuntos[difuntos["FECHA_ARCHIVO"]==fecha_pan]
    matriz_filt = geomatriz(filtrados,"FECHA_SINTOMAS",pacientes_edo,varios_edo,pacientes_mun,varios_mun)
    matriz_filt.cumsum().to_csv("./datos_procesados/defun_sint_" + fecha_pan.strftime("%d-%m") + ".csv")